In [23]:
import gzip
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [24]:
user_purchased_category = defaultdict(list)
item_belong_to_category = defaultdict(list)

count = 0
for l in readGz("train.json.gz"):
  for i in l['categories'] :
     if not i in item_belong_to_category[l['itemID']] :
           item_belong_to_category[l['itemID']].append(i)
     if not i in user_purchased_category[l['reviewerID']] :
           user_purchased_category[l['reviewerID']].append(i) 

In [25]:
businessCount = defaultdict(int)
totalPurchases = 0
count=0

###  all user ID and item ID in data
user_set = []
item_set = []

### already purchased data for validation ( should predicted positive )
validation_set1 = defaultdict(list)
user_items = defaultdict(list)
i_be_bought = defaultdict(set)
u_bought = defaultdict(set)

for l in readGz("train.json.gz"):
  count+=1
  if not l['reviewerID'] in user_set :
        user_set.append(l['reviewerID'])
  if not l['itemID'] in item_set :
        item_set.append(l['itemID'])
  user_items[l['reviewerID']].append(l['itemID'])     
#  if count<=100000 :
  user,business = l['reviewerID'],l['itemID']
  businessCount[business] += 1
  totalPurchases += 1    
  i_be_bought[l['itemID']].add((l['reviewerID']))
  u_bought[l['reviewerID']].add((l['itemID']))

In [47]:
mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

### return "recommend-purchase" according to their popular (bought times)
return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPurchases*0.4: break     ######## can change threshold

In [27]:
def is_category(user,item):
    user_set = user_purchased_category[user]
    item_set = item_belong_to_category[item]
    for cat in item_set :
        if cat in user_set :
            return True
    return False

In [28]:
## part 3
def jaccard(u,v) :
    set1 = u_bought[u]
    set2 = u_bought[v]
    if len(set1 | set2)==0 :
        return 0
    return len(set1 & set2)/len(set1 | set2)

def jaccard_u(i,u) :
    u_buy_i = i_be_bought[i]
    sim = 0
    for t in u_buy_i :
        sim += jaccard(t,u)
    if sim == 0:
        return 0
    else : return 1

In [48]:
### see the accuracy of baseline model on the text set
count = 0
predictions = open("predictions_purchase_base_11.18_22.05.txt", 'w')
for l in open("pairs_Purchase.txt"):
  if l.startswith("reviewerID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  same_cat = is_category(u,i)
  if i in return1 or same_cat or jaccard_u(i,u):
    predictions.write(u + '-' + i + ',' + str(1) + '\n')
    count += 1
  else :
    predictions.write(u + '-' + i + ',' + str(0) + '\n')

predictions.close()